  # RAG v1 — Base Template

This notebook is a starting point for a Retrieval-Augmented Generation (RAG) app.

## Goals
- Ingest a small document set
- Create embeddings + vector index
- Build a retriever + answer generation pipeline
- Evaluate with simple checks

## Environment setup (recommended)
Create a local virtual env, install dependencies, and register a Jupyter kernel:

```bash
uv venv .venv
source .venv/bin/activate
uv python -m pip install --upgrade pip
uv pip install jupyter ipykernel \
  langchain langchain-community langchain-openai langchain-google-genai \
  langchain-nvidia-ai-endpoints \
  faiss-cpu sentence-transformers python-dotenv requests certifi \
  beautifulsoup4 markdownify lxml
uv run python -m ipykernel install --user --name .venv --display-name ".venv"
```

In Jupyter, select the kernel: `.venv`.

## Environment variables
Add keys to a `.env` file at the project root (do NOT commit this file):

```bash
# .env
BINDING=nvidia
API_KEY=your_binding_key  
BINDING_HOST=https://integrate.api.nvidia.com/v1

# 
EMBEDDING_MODEL=baai/bge-m3
EMBEDDING_DIM=1024
EMBEDDING_SEND_DIM=false
EMBEDDING_TOKEN_LIMIT=8192

# 
LLM_MODEL=meta/llama-3.3-70b-instruct
```

## Notes
- Keep secrets out of the notebook; use environment variables instead.
- Replace placeholders with your data sources and preferred models.


Imports and Envs

In [1]:
from __future__ import annotations

import os
from dataclasses import dataclass
from pathlib import Path
from typing import List

from dotenv import  load_dotenv

PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

# Load environment variables from .env (if present)
env_path = PROJECT_ROOT / ".env"
load_dotenv(env_path, override=True)


@dataclass
class RagConfig:
    chunk_size: int = 800
    chunk_overlap: int = 120
    top_k: int = 10

config = RagConfig()

BINDING = os.getenv("BINDING", "").strip().lower()
BINDING_HOST = os.getenv("BINDING_HOST", "").strip()
API_KEY = os.getenv("API_KEY", "").strip()

EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL", "").strip()
EMBEDDING_DIM = os.getenv("EMBEDDING_DIM", "").strip()
EMBEDDING_SEND_DIM = os.getenv("EMBEDDING_SEND_DIM", "").strip().lower()
EMBEDDING_TOKEN_LIMIT = os.getenv("EMBEDDING_TOKEN_LIMIT", "").strip()

LLM_MODEL = os.getenv("LLM_MODEL", "").strip()

# Fallback: set directly here if your .env isn't loading yet
# LLM_BINDING_API_KEY = "your_binding_key"

if not API_KEY:
    raise RuntimeError(
        "Set API_KEY"
    )

# LLM provider (for generation)
if BINDING:
    PROVIDER = BINDING
else:
    PROVIDER = "openai"

print(f"Using LLM provider: {PROVIDER}")


Using LLM provider: nvidia


## LOADING AND PREPROCESSING

In [ ]:
# # Create dummy data — HTML files in data/html/ to mimic real input

# HTML_DIR = DATA_DIR / "html"
# HTML_DIR.mkdir(exist_ok=True)

# sample_html_docs = {
#     "trading_basics.html": """<!DOCTYPE html>
# <html><body>
# <h1>Trading Basics</h1>
# <p>Trading involves buying and selling financial instruments such as stocks,
# bonds, commodities, or derivatives to generate profit.</p>

# <h2>Types of Trading</h2>
# <p>Common trading styles include day trading, swing trading, and position trading.
# Each has different time horizons and risk profiles.</p>

# <h3>Day Trading</h3>
# <p>Day traders open and close positions within a single trading session.
# They rely on technical analysis, chart patterns, and real-time data feeds.</p>

# <h3>Swing Trading</h3>
# <p>Swing traders hold positions for several days to weeks, aiming to capture
# short- to medium-term price movements using both technical and fundamental analysis.</p>

# <h2>Key Concepts</h2>
# <p>Understanding support and resistance levels, moving averages, and volume
# indicators is essential for any trading strategy.</p>
# </body></html>""",

#     "risk_management.html": """<!DOCTYPE html>
# <html><body>
# <h1>Risk Management in Trading</h1>
# <p>Effective risk management is the single most important factor that separates
# successful traders from unsuccessful ones.</p>

# <h2>Position Sizing</h2>
# <p>Never risk more than 1-2% of your total account balance on a single trade.
# This ensures that a series of losses does not wipe out your capital.</p>

# <h2>Stop-Loss Orders</h2>
# <p>A stop-loss order automatically closes a position when the price moves against
# you by a specified amount. It limits downside risk on every trade.</p>

# <h3>Trailing Stop</h3>
# <p>A trailing stop adjusts automatically as the price moves in your favour,
# locking in profits while still providing downside protection.</p>

# <h2>Risk-Reward Ratio</h2>
# <p>Aim for a minimum risk-reward ratio of 1:2. This means your potential profit
# should be at least twice your potential loss on each trade.</p>
# </body></html>""",

#     "technical_indicators.html": """<!DOCTYPE html>
# <html><body>
# <h1>Technical Indicators</h1>
# <p>Technical indicators are mathematical calculations based on price, volume,
# or open interest data. They help traders identify trends and potential reversals.</p>

# <h2>Moving Averages</h2>
# <p>The Simple Moving Average (SMA) and Exponential Moving Average (EMA) smooth
# out price data to reveal the underlying trend direction.</p>

# <h2>Relative Strength Index (RSI)</h2>
# <p>RSI is a momentum oscillator that measures the speed and magnitude of price
# changes. Values above 70 suggest overbought conditions; below 30 suggest oversold.</p>

# <h2>MACD</h2>
# <p>The Moving Average Convergence Divergence indicator shows the relationship
# between two moving averages. A MACD crossover can signal a trend change.</p>

# <h2>Bollinger Bands</h2>
# <p>Bollinger Bands consist of a middle SMA band and two standard-deviation bands.
# Price touching the outer bands can indicate overbought or oversold conditions.</p>
# </body></html>""",
# }

# existing_html = list(HTML_DIR.rglob("*.html"))
# if not existing_html:
#     for filename, content in sample_html_docs.items():
#         (HTML_DIR / filename).write_text(content.strip(), encoding="utf-8")
#     print(f"Wrote {len(sample_html_docs)} dummy HTML files to {HTML_DIR}")
# else:
#     print(f"HTML folder already has {len(existing_html)} file(s); skipping dummy data.")


Wrote 3 dummy HTML files to /Users/siddhartha/Desktop/PROJECTS/ai-trader-education-bot/data/html


In [10]:
## ------ CRAWL AND SCRAPE CONTENT FROM WEBPAGE ------ ##

import time
import requests
import certifi
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup

# ── Content source definitions ───────────────────────────────────────
# Each source: index pages to crawl, URL path patterns to match,
# canonical URL template (with {slug}), and output subdirectory.
import json

CONTENT_SOURCES = [
    {
        "name": "blog",
        "index_urls": [
            "https://deriv.com/blog",
            "https://deriv.com/blog-categories/market-news",
            "https://deriv.com/blog-categories/trading-strategies",
        ],
        "path_patterns": ["/blog/posts/", "/blog-posts/"],
        "canonical_tpl": "https://deriv.com/blog/posts/{slug}",
        "output_dir": "blog",
    },
    {
        "name": "experts",
        "index_urls": [
            "https://experts.deriv.com/insights",
        ],
        "path_patterns": ["/insights/"],
        "canonical_tpl": "https://experts.deriv.com/insights/{slug}",
        "output_dir": "experts",
    },
    {
        "name": "guides",
        "index_urls": [
            "https://traders-academy.deriv.com/trading-guides",
        ],
        "path_patterns": ["/trading-guides/"],
        "canonical_tpl": "https://traders-academy.deriv.com/trading-guides/{slug}",
        "output_dir": "guides",
    },
]

RATE_LIMIT_SECS = 1.5
URL_MAP_PATH = DATA_DIR / "url_map.json"


# ── HTTP session with browser-like headers ───────────────────────────
_session = requests.Session()
_session.headers.update({
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0.0.0 Safari/537.36"
    ),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
})
_session.verify = certifi.where()


def fetch_page(url: str, retries: int = 2, delay: float = 1.0) -> str | None:
    """Fetch a URL and return raw HTML, with simple retry + back-off."""
    for attempt in range(retries + 1):
        try:
            resp = _session.get(url, timeout=30)
            resp.raise_for_status()
            return resp.text
        except requests.RequestException as e:
            print(f"  [attempt {attempt + 1}/{retries + 1}] {url}: {e}")
            if attempt < retries:
                time.sleep(delay * (attempt + 1))
    return None


# Slugs to skip (index / auth / utility pages, not articles)
SKIP_SLUGS = {
    "", "login", "signup", "open-account", "appstore", "traders-hub",
    "all-articles", "all-videos", "blog", "insights",
    "trading-guides", "trading-courses", "experts",
}


def extract_article_urls(
    html: str,
    base_url: str,
    path_patterns: list[str],
    canonical_tpl: str,
) -> dict[str, str]:
    """Extract {slug: canonical_url} from an index page.

    Matches links whose path contains any of *path_patterns*.
    Normalises to canonical form using *canonical_tpl*.
    Returns dict mapping slug -> canonical URL.
    """
    soup = BeautifulSoup(html, "lxml")
    results: dict[str, str] = {}

    for a_tag in soup.find_all("a", href=True):
        href = a_tag["href"]
        full = urljoin(base_url, href)
        parsed = urlparse(full)
        path = parsed.path.rstrip("/")

        for pattern in path_patterns:
            if pattern.rstrip("/") in path:
                slug = path.split("/")[-1]
                if slug and slug not in SKIP_SLUGS and not slug.startswith("#"):
                    results[slug] = canonical_tpl.format(slug=slug)
                break

    return results


# ── Load existing URL map ────────────────────────────────────────────
if URL_MAP_PATH.exists():
    url_map: dict = json.loads(URL_MAP_PATH.read_text(encoding="utf-8"))
else:
    url_map: dict = {}


# ── Crawl all content sources ────────────────────────────────────────
total_new, total_skip, total_fail = 0, 0, 0

for source in CONTENT_SOURCES:
    name = source["name"]
    out_dir = DATA_DIR / "html" / source["output_dir"]
    out_dir.mkdir(parents=True, exist_ok=True)

    print(f"\n{'─' * 50}")
    print(f"Source: {name}")
    print(f"{'─' * 50}")

    # Step 1: Discover article URLs from index pages
    discovered: dict[str, str] = {}  # slug -> canonical URL
    for idx_url in source["index_urls"]:
        html = fetch_page(idx_url)
        if html:
            found = extract_article_urls(
                html, idx_url,
                source["path_patterns"],
                source["canonical_tpl"],
            )
            discovered.update(found)
            print(f"  {idx_url} -> {len(found)} links")
        else:
            print(f"  {idx_url} -> FAILED")

    print(f"  Unique articles: {len(discovered)}\n")

    # Step 2: Scrape & save each article
    already_saved = {p.stem for p in out_dir.glob("*.html")}
    new_count, skip_count, fail_count = 0, 0, 0

    for slug, canonical_url in sorted(discovered.items()):
        # Always register in url_map
        url_map[f"{slug}.html"] = {"url": canonical_url, "source": name}

        if slug in already_saved:
            skip_count += 1
            continue

        print(f"  Scraping: {slug} ...", end=" ")
        article_html = fetch_page(canonical_url)

        if article_html:
            (out_dir / f"{slug}.html").write_text(article_html, encoding="utf-8")
            size_kb = len(article_html) / 1024
            print(f"OK ({size_kb:.1f} KB)")
            new_count += 1
        else:
            print("FAILED")
            fail_count += 1

        time.sleep(RATE_LIMIT_SECS)

    total_new += new_count
    total_skip += skip_count
    total_fail += fail_count

    total_files = len(list(out_dir.glob("*.html")))
    print(f"  New: {new_count} | Cached: {skip_count} | Failed: {fail_count} | Total: {total_files}")


# ── Persist URL map ──────────────────────────────────────────────────
URL_MAP_PATH.write_text(
    json.dumps(url_map, indent=2, ensure_ascii=False),
    encoding="utf-8",
)

print(f"\n{'=' * 50}")
print(f"TOTAL — New: {total_new}  Cached: {total_skip}  Failed: {total_fail}")
print(f"URL map: {len(url_map)} entries -> {URL_MAP_PATH.relative_to(PROJECT_ROOT)}")


Discovering article URLs from seed pages ...

  https://deriv.com/blog
    -> 19 article links

  https://deriv.com/blog-categories/market-news
    -> 10 article links

  https://deriv.com/blog-categories/trading-strategies
    -> 10 article links

Total unique article URLs: 31

  Scraping: 2025-holiday-trading-hours ... OK (266.6 KB)
  Scraping: amazon-ai-spending-warning-or-opportunity ... OK (222.7 KB)
  Scraping: become-a-copy-trading-strategy-provider ... OK (218.3 KB)
  Scraping: bitcoin-crash-analysts-doubt-80-percent-drop ... OK (220.6 KB)
  Scraping: bitcoin-drop-market-shift ... OK (220.3 KB)
  Scraping: bitcoin-test-sell-pressure-eases ... OK (221.7 KB)
  Scraping: decline-in-the-stock-market ... OK (223.3 KB)
  Scraping: deriv-achieves-prestigious-platinum-accreditation-by-investors-in-people ... OK (206.1 KB)
  Scraping: earnings-analysis-netflix-meta-microsoft ... OK (209.4 KB)
  Scraping: five-reasons-why-gold-acts-as-a-hedge-against-inflation ... OK (214.8 KB)
  Scrapin

In [11]:
## ------ CONVERTING HTML TO MARKDOWN ------ ##

import json as _json
import re

from bs4 import BeautifulSoup
from langchain_core.documents import Document
from markdownify import markdownify as md

# Create markdown output folder matching the directory structure of HTML input
HTML_ROOT_DIR = DATA_DIR / "html"
MARKDOWN_ROOT_DIR = DATA_DIR / "markdown"
MARKDOWN_ROOT_DIR.mkdir(parents=True, exist_ok=True)

# Load URL map for source-URL resolution
_url_map_path = DATA_DIR / "url_map.json"
_url_map: dict = {}
if _url_map_path.exists():
    _url_map = _json.loads(_url_map_path.read_text(encoding="utf-8"))

def html_to_clean_markdown(html: str) -> str:
    """Convert raw HTML to clean markdown, stripping scripts/styles/nav."""
    soup = BeautifulSoup(html, "lxml")

    # Remove non-content tags
    for tag in soup(["script", "style", "nav", "footer", "header", "aside", "form"]):
        tag.decompose()

    # Convert remaining HTML to markdown
    markdown = md(str(soup), heading_style="ATX", strip=["img", "a"])

    # Collapse excessive blank lines
    markdown = re.sub(r"\n{3,}", "\n\n", markdown).strip()
    return markdown


def html_path_to_markdown_path(html_path: Path) -> Path:
    """Given a Path under the HTML root, compute matching markdown Path under markdown root."""
    rel_path = html_path.relative_to(HTML_ROOT_DIR)
    md_path = MARKDOWN_ROOT_DIR / rel_path.with_suffix(".md")
    md_path.parent.mkdir(parents=True, exist_ok=True)
    return md_path

def load_html_documents(folder: Path) -> list[Document]:
    """
    Load all .html/.htm files from folder (recursively), convert to markdown Documents.
    Save each markdown result to the equivalent path under data/markdown/ (mirroring structure).
    """
    docs: list[Document] = []
    for path in sorted(folder.rglob("*.htm*")):
        raw_html = path.read_text(encoding="utf-8", errors="ignore")
        markdown_text = html_to_clean_markdown(raw_html)
        if not markdown_text:
            continue

        # Write markdown to equivalent path in markdown/ tree
        md_path = html_path_to_markdown_path(path)
        md_path.write_text(markdown_text, encoding="utf-8")
        # Resolve source URL from url_map (falls back to filename)
        source_url = ""
        if path.name in _url_map:
            source_url = _url_map[path.name].get("url", "")

        docs.append(
            Document(
                page_content=markdown_text,
                metadata={
                    "source": str(path),
                    "filename": path.name,
                    "source_url": source_url,
                    "format": "html->markdown",
                    "markdown_path": str(md_path),
                },
            )
        )
    return docs

raw_docs = load_html_documents(HTML_ROOT_DIR)
print(f"Loaded {len(raw_docs)} HTML documents (converted to markdown)")
for doc in raw_docs:
    print(f"  - {doc.metadata['filename']}  ({len(doc.page_content)} chars, saved: {doc.metadata.get('markdown_path', '')})")


/var/folders/3c/jkzpw6zd7_71d_kpydtz_01m0000gn/T/ipykernel_67040/1917982555.py:16: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(html, "lxml")


Loaded 34 HTML documents (converted to markdown)
  - 2025-holiday-trading-hours.html  (24912 chars, saved: /Users/siddhartha/Desktop/PROJECTS/ai-trader-education-bot/data/markdown/blog/2025-holiday-trading-hours.md)
  - amazon-ai-spending-warning-or-opportunity.html  (9986 chars, saved: /Users/siddhartha/Desktop/PROJECTS/ai-trader-education-bot/data/markdown/blog/amazon-ai-spending-warning-or-opportunity.md)
  - become-a-copy-trading-strategy-provider.html  (7391 chars, saved: /Users/siddhartha/Desktop/PROJECTS/ai-trader-education-bot/data/markdown/blog/become-a-copy-trading-strategy-provider.md)
  - bitcoin-crash-analysts-doubt-80-percent-drop.html  (6675 chars, saved: /Users/siddhartha/Desktop/PROJECTS/ai-trader-education-bot/data/markdown/blog/bitcoin-crash-analysts-doubt-80-percent-drop.md)
  - bitcoin-drop-market-shift.html  (7790 chars, saved: /Users/siddhartha/Desktop/PROJECTS/ai-trader-education-bot/data/markdown/blog/bitcoin-drop-market-shift.md)
  - bitcoin-test-sell-pressure

## CHUNKING

In [12]:
# Two-stage splitting:
# 1. MarkdownHeaderTextSplitter — split on headers to preserve section context
# 2. RecursiveCharacterTextSplitter — sub-split large sections to fit embedding limits

from langchain_text_splitters import (
    MarkdownHeaderTextSplitter,
    RecursiveCharacterTextSplitter,
)

# Headers to split on (markdown ATX headings)
headers_to_split_on = [
    ("#", "h1"),
    ("##", "h2"),
    ("###", "h3"),
]

md_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on,
    strip_headers=False,  # keep headers in chunk text for context
)

char_splitter = RecursiveCharacterTextSplitter(
    chunk_size=config.chunk_size,
    chunk_overlap=config.chunk_overlap,
)

chunks = []
for doc in raw_docs:
    # Stage 1: header-aware split
    header_chunks = md_splitter.split_text(doc.page_content)

    for hc in header_chunks:
        # Carry over source metadata from the original document
        hc.metadata = {**doc.metadata, **hc.metadata}

    # Stage 2: sub-split any section that exceeds chunk_size
    sub_chunks = char_splitter.split_documents(header_chunks)
    chunks.extend(sub_chunks)

print(f"Built {len(chunks)} chunks from {len(raw_docs)} documents")
print(f"Avg chunk size: {sum(len(c.page_content) for c in chunks) // max(len(chunks), 1)} chars")

# Preview first few chunks
for i, chunk in enumerate(chunks[:3]):
    hdrs = {k: v for k, v in chunk.metadata.items() if k.startswith("h")}
    print(f"\n--- Chunk {i} | headers={hdrs} ---")
    print(chunk.page_content[:200])


Built 555 chunks from 34 documents
Avg chunk size: 457 chars

--- Chunk 0 | headers={} ---
2025 year-end holiday trading blog: Market hours and holiday calendar  
HomeBlog  
Trading strategies  
2025 year-end holiday trading blog (holiday calendar)

--- Chunk 1 | headers={'h1': '2025 year-end holiday trading blog (holiday calendar)'} ---
# 2025 year-end holiday trading blog (holiday calendar)  
December 9, 2025  
***Disclaimer****: Trading hours listed on this blog are for reference only and may differ due to last-minute changes.*  
A

--- Chunk 2 | headers={'h1': '2025 year-end holiday trading blog (holiday calendar)'} ---
The holiday season brings its own rhythm to the markets, and knowing when things are open, closed, or moving a little differently can make all the difference. Whether you’re trading full-time or simpl


## EMBEDDING

In [13]:

from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

import os

embeddings = NVIDIAEmbeddings(
    model="baai/bge-m3",
    api_key=os.getenv("API_KEY"),
    base_url="https://integrate.api.nvidia.com/v1",
)

print("Storing embeddings")

vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)
print("Vector store ready" if vectorstore else "No chunks to index")


Storing embeddings
Vector store ready


In [14]:
# Verify embeddings were stored

if vectorstore is None:
    print("Vectorstore is None - no embeddings stored.")
else:
    try:
        total = vectorstore.index.ntotal  # FAISS internal count
    except Exception:
        total = "unknown"

    print(f"Vectorstore contains {total} vectors")

    # Quick sanity check retrieval
    test_query = "What is retrieval-augmented generation?"
    results = vectorstore.similarity_search(test_query, k=2)
    print(f"Retrieved {len(results)} docs for sanity check")
    for i, doc in enumerate(results, start=1):
        source = doc.metadata.get("source", "unknown")
        print(f"[{i}] {source}\n{doc.page_content[:200]}\n")


Vectorstore contains 555 vectors
Retrieved 2 docs for sanity check
[1] /Users/siddhartha/Desktop/PROJECTS/ai-trader-education-bot/data/html/blog/google-stock-rally-alphabet-3-trillion-path.html
What are Alphabet’s main growth drivers?  
Alphabet has four major growth engines. Google Cloud is the standout, growing at 32 percent year over year, outpacing rivals. Search continues to be highly p

[2] /Users/siddhartha/Desktop/PROJECTS/ai-trader-education-bot/data/html/blog/amazon-ai-spending-warning-or-opportunity.html
That said, Amazon is not merely a buyer in the AI ecosystem. Through Annapurna Labs, it has developed a substantial in-house chip business. Custom processors such as Trainium and Graviton now generate



## QUERYING

In [15]:
SYSTEM_PROMPT = """\
You are a **trading knowledge assistant** embedded in an agentic trade-analysis pipeline.
Your consumer is another AI agent that analyses charts, indicator values, and market \
conditions to make trade decisions. Your job is to provide knowledge-grounded answers \
that the agent can act on directly.

## Inputs you receive
| Input            | Description |
|-----------------|-------------|
| TRADE_ANALYSIS  | The upstream agent's trade context. This can be a **pre-trade** chart analysis (asset, indicators, chart patterns, proposed action) OR a **post-trade** review (trade outcome, entry/exit points, payout, learning observations). May be "none" if not provided. |
| QUESTION        | The specific knowledge question the agent needs answered. |
| CONTEXT         | Retrieved knowledge-base chunks relevant to the question. |

## Internal reasoning (do NOT output — perform silently before answering)
Step 1 — **Assess difficulty** of the QUESTION:
  - `beginner`: definitional, single-concept ("What is RSI?")
  - `intermediate`: applied interpretation requiring thresholds or rules ("How do I read a MACD divergence?")
  - `advanced`: multi-factor synthesis across technical + fundamental signals ("How do macro indicators interact with chart patterns?")
Step 2 — **Filter context**: identify which CONTEXT chunks are relevant to BOTH the QUESTION and the TRADE_ANALYSIS. Silently discard unrelated chunks.
Step 3 — **Anchor to scenario**: if TRADE_ANALYSIS is provided, weave the specific trade details (asset, trade type, indicator values, entry/exit points, outcome) into your answer so it is directly applicable to the agent's situation — whether it is a pre-trade setup or a post-trade review.

## Output schema (strict JSON)
{format_instructions}

## Response-depth guidelines

| Difficulty    | Target length    | Style |
|--------------|------------------|-------|
| beginner     | 2–4 sentences    | Plain language. Define any jargon. One clear, actionable takeaway. |
| intermediate | 1–2 paragraphs   | Specific thresholds, rules, and bullet points. Apply directly to the trade scenario. |
| advanced     | 2–4 paragraphs   | Nuanced analysis with caveats and trade-offs. Cross-reference multiple sources. Discuss interactions between factors. |

## Tone & trader-psychology guidelines (avoid defensiveness)
- Use neutral, non-blaming language, especially for losing trades.
- Focus on **process**, **evidence**, and **next steps**, not personal judgment.
- Frame feedback as observations and options: "one possible improvement" / "consider".
- Acknowledge what went well before describing improvements (if supported by context).
- Avoid shaming phrases ("you should have known", "obvious mistake").

## Rules
1. Use ONLY information present in the CONTEXT. Never use prior training knowledge.
2. If CONTEXT is insufficient to answer, return the JSON object with:
   - `answer`: "INSUFFICIENT_CONTEXT"
   - `confidence`: "low"
   - `sources`: []
3. When TRADE_ANALYSIS is provided, anchor every point to that exact scenario — \
reference the specific asset, trade type, entry/exit points, indicator values, outcomes, and conditions mentioned.
4. When TRADE_ANALYSIS is "none", answer generically from context alone.
5. Cite specific numbers, thresholds, percentages, and definitions from context wherever available.
6. Never issue a direct buy/sell recommendation. Present what the evidence suggests \
and let the calling agent make the final decision.
7. If the question is ambiguous, answer the most likely interpretation and note the ambiguity briefly.

## Few-shot examples

### Example 1 — Beginner (post-trade review)
TRADE_ANALYSIS: Great job on this CALL option trade! You correctly identified that the price would increase within the 5-minute timeframe, turning a $10 investment into a $19.50 payout. Entry tick was 250.50, closed at 251.00. Learning Points: CALL options are suitable when you anticipate a price increase. Defined buy prices help manage risk.
QUESTION: What are CALL options and when are they the right choice?
CONTEXT:
[trading_basics.html]
Trading involves buying and selling financial instruments such as stocks, bonds, commodities, or derivatives to generate profit. Common trading styles include day trading, swing trading, and position trading.
[risk_management.html]
Never risk more than 1-2% of your total account balance on a single trade. This ensures that a series of losses does not wipe out your capital.

{"difficulty":"beginner","answer":"A CALL option is a contract that profits when the asset's price rises above your entry point — exactly what happened in your trade where price moved from 250.50 to 251.00. They are the right choice when you anticipate upward price movement within a defined timeframe. Your $10 defined buy price is also good risk management practice, keeping single-trade exposure controlled (the knowledge base recommends risking no more than 1–2% of account balance per trade).","confidence":"high","sources":["trading_basics.html","risk_management.html"]}

### Example 2 — Intermediate
TRADE_ANALYSIS: Gold spot at $4,860. RSI = 48 (neutral). Bollinger Bands expanding after multi-day consolidation. Evaluating a long entry on pullback.
QUESTION: How do Bollinger Bands help identify potential entry points?
CONTEXT:
[technical_indicators.html]
Bollinger Bands consist of a middle SMA band and two standard-deviation bands. Price touching the outer bands can indicate overbought or oversold conditions.
[https://deriv.com/blog/posts/gold-prices-fall-jobless-claims]
Gold has consolidated after a sharp advance, with price oscillating around $4,850. Bollinger Bands remain widely expanded, indicating elevated volatility.

{"difficulty":"intermediate","answer":"Bollinger Bands plot a middle SMA with upper/lower bands at two standard deviations. Key signals for your gold setup: (1) Band expansion at $4,860 indicates rising volatility — breakout probability increases, but so does risk of false moves. (2) Price near the lower band during an uptrend can signal a pullback entry; near the upper band may signal overextension. (3) Narrowing bands (squeeze) typically precede a strong directional breakout. In your scenario, gold oscillating at $4,860 with expanding bands and a neutral RSI (48) suggests the market is transitioning out of consolidation. The knowledge base confirms elevated volatility at this price level following a prior rally, consistent with a potential breakout setup — but the neutral RSI means directional conviction is still building.","confidence":"high","sources":["technical_indicators.html","https://deriv.com/blog/posts/gold-prices-fall-jobless-claims"]}

### Example 3 — Advanced
TRADE_ANALYSIS: S&P 500 daily. Death cross confirmed (50 SMA < 200 SMA). RSI = 42. Volume declining on rallies. Fed holding rates. Weekly jobless claims at 231K.
QUESTION: How do deteriorating macro indicators interact with bearish technical signals for index analysis?
CONTEXT:
[https://deriv.com/blog/posts/sp-500-outlook-tight-liquidity-impact]
The S&P 500 is still holding near record levels, yet beneath the surface, market liquidity is tightening in ways that historically make equity rallies harder to sustain.
[https://deriv.com/blog/posts/gold-prices-fall-jobless-claims]
Labour market trends directly influence rate expectations. Job openings fell to 6.54 million in December, the lowest since September 2020. Jobless claims jumped to 231,000, the largest weekly increase since early December.

{"difficulty":"advanced","answer":"The confluence of bearish technicals and weakening macro data in your S&P 500 scenario warrants layered analysis. Technical structure: the confirmed death cross with RSI at 42 and declining volume on bounces forms a bearish triad. Falling volume on rallies indicates weakening buying conviction — a pattern that historically precedes further downside or extended sideways consolidation. Macro backdrop: the knowledge base notes S&P 500 liquidity is tightening beneath the surface, making sustained rallies harder to achieve. Jobless claims have spiked to 231,000 (highest in two months, well above the 212,000 forecast), and job openings hit a five-year low at 6.54 million. Analysts describe a low-hire, low-fire dynamic — suggesting economic cooling rather than outright collapse. Synthesis: when bearish technical signals align with deteriorating macro fundamentals (tightening liquidity, rising claims), historical precedent favours a defensive posture. The Fed's decision to hold rates removes any near-term dovish catalyst. Monitor for a volume spike on the downside as confirmation, or an RSI drop below 30 as an oversold counter-signal.","confidence":"high","sources":["https://deriv.com/blog/posts/sp-500-outlook-tight-liquidity-impact","https://deriv.com/blog/posts/gold-prices-fall-jobless-claims"]}

### Example 4 — Insufficient context
TRADE_ANALYSIS: Reviewing DOGE/USDT perpetual futures on Binance.
QUESTION: Which exchange offers the best liquidity for DOGE perpetual futures?
CONTEXT:
[trading_basics.html]
Trading involves buying and selling financial instruments such as stocks, bonds, commodities, or derivatives to generate profit.

{"difficulty":"intermediate","answer":"INSUFFICIENT_CONTEXT","confidence":"low","sources":[]}
"""

In [16]:
# --- Retriever ---
retriever = vectorstore.as_retriever(search_kwargs={"k": config.top_k}) if vectorstore else None


from typing import Literal

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from pydantic import BaseModel, Field


class RagAnswer(BaseModel):
    difficulty: Literal["beginner", "intermediate", "advanced", "unknown"] = Field(
        description="Difficulty level inferred from the question"
    )
    answer: str = Field(description="Knowledge-grounded answer or INSUFFICIENT_CONTEXT")
    confidence: Literal["high", "medium", "low"] = Field(
        description="Confidence based on context coverage"
    )
    sources: list[str] = Field(description="Source URLs (or filenames when URL unavailable)")


parser = PydanticOutputParser(pydantic_object=RagAnswer)

# --- LLM ---
if BINDING and BINDING_HOST and API_KEY:
    from langchain_openai import ChatOpenAI

    llm = ChatOpenAI(
        model=LLM_MODEL or "meta/llama-3.3-70b-instruct",
        api_key=API_KEY,
        base_url=BINDING_HOST,
        temperature=0,
    )
else:
    from langchain_openai import ChatOpenAI

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_PROMPT),
        (
            "human",
            "TRADE_ANALYSIS:\n{trade_analysis}\n\n"
            "QUESTION:\n{question}\n\n"
            "CONTEXT:\n{context}",
        ),
    ]
).partial(format_instructions=parser.get_format_instructions())


def format_docs(docs: List) -> str:
    """Format retrieved documents with source labels for the prompt.

    Prefers source_url (the original web link) over filename for
    each chunk label so the LLM cites URLs in its response.
    """
    return "\n\n".join(
        f"[{doc.metadata.get('source_url') or doc.metadata.get('filename', 'unknown')}]\n{doc.page_content}"
        for doc in docs
    )


def _as_dict(result: RagAnswer) -> dict:
    """Return a stable dict across Pydantic v1/v2."""
    if hasattr(result, "model_dump"):
        return result.model_dump()
    return result.dict()


def answer_question(question: str, trade_analysis: str = "none") -> dict:
    """Query the RAG pipeline with an optional trade-analysis scenario.

    Args:
        question: The knowledge question to answer.
        trade_analysis: Upstream agent's trade scenario (asset, indicators,
                        chart state, proposed action). Pass "none" or omit
                        for a generic knowledge lookup.
    Returns:
        Parsed JSON response with keys: difficulty, answer, confidence, sources.
    """
    if retriever is None:
        return _as_dict(
            RagAnswer(
                difficulty="unknown",
                answer="INSUFFICIENT_CONTEXT",
                confidence="low",
                sources=[],
            )
        )

    # Retrieve context chunks based on the question
    docs = retriever.invoke(question)
    context = format_docs(docs)

    chain = prompt | llm | parser

    result = chain.invoke({
        "trade_analysis": trade_analysis,
        "question": question,
        "context": context,
    })

    return _as_dict(result)


In [17]:
# ── Test 1: Beginner — post-trade CALL option review ─────────────────
print("TEST 1 — Beginner + post-trade CALL option analysis")
print("-" * 50)
answer1 = answer_question(
    question="What are CALL options and when should they be used?",
    trade_analysis=(
        "Great job on this CALL option trade! You correctly identified that the "
        "price of the asset would increase within the 5-minute timeframe, turning "
        "a $10 investment into a $19.50 payout. Your entry tick was 250.50, and you "
        "closed the trade at 251.00, confirming your prediction. This win highlights "
        "the importance of accurately forecasting price movements and selecting the "
        "right contract type to match your market outlook.\n\n"
        "CALL options are excellent tools when you anticipate an asset's price will "
        "rise. By setting a defined buy price ($10 in this case), you're also "
        "practicing effective risk management. Even though this trade was profitable, "
        "remember that every trade carries risk. Keep analyzing market trends, "
        "refining your prediction skills, and sticking to your risk management strategy.\n\n"
        "Learning Points:\n"
        "- CALL options are suitable when you anticipate a price increase.\n"
        "- Accurate price movement prediction is crucial for profitable trading.\n"
        "- Defined buy prices help manage risk in options trading."
    ),
)
print(answer1)
print("\n" + "=" * 60 + "\n")

# ── Test 2: Intermediate — risk management with active trade context ─
print("TEST 2 — Intermediate + pre-trade risk management")
print("-" * 50)
answer2 = answer_question(
    question="How should I set my stop-loss and what position size is appropriate?",
    trade_analysis=(
        "Asset: Gold (XAU/USD) | Account balance: $5,000 | "
        "Entry price: $4,860 | Support level: $4,820 | "
        "Considering long position after Bollinger Band bounce."
    ),
)
print(answer2)
print("\n" + "=" * 60 + "\n")

# ── Test 3: Advanced — macro + technical synthesis ───────────────────
print("TEST 3 — Advanced + multi-factor analysis")
print("-" * 50)
answer3 = answer_question(
    question="How do deteriorating macro indicators interact with my bearish chart signals?",
    trade_analysis=(
        "S&P 500 daily chart. Death cross confirmed (50 SMA crossed below 200 SMA). "
        "RSI = 42 and flat. Volume declining on every rally attempt. "
        "Fed held rates steady last week. US weekly jobless claims at 231K. "
        "Considering reducing equity exposure."
    ),
)
print(answer3)
print("\n" + "=" * 60 + "\n")

# ── Test 4: Generic — no trade analysis provided ────────────────────
print("TEST 4 — Generic knowledge query (no trade analysis)")
print("-" * 50)
answer4 = answer_question(
    question="What are the key differences between a trailing stop and a fixed stop-loss?",
)
print(answer4)
print("\n" + "=" * 60 + "\n")

# ── Test 5: Generic — trade-analysis-only queries ───────────────────
print("TEST 5 — Generic trade-analysis-only queries")
print("-" * 50)
trade_review = (
    "CALL option trade review: 5-minute expiry. Entry tick 250.50, exit 251.00. "
    "$10 stake, $19.50 payout. Rationale: expected short-term upside after a small "
    "bounce from support. Learning points: matching contract type to direction, "
    "defined stake limits risk."
)

answer5a = answer_question(
    question="Summarize what went well and what should be improved in this trade.",
    trade_analysis=trade_review,
)
print("(A)")
print(answer5a)
print("\n" + "-" * 50 + "\n")

answer5b = answer_question(
    question="Was risk management handled appropriately in this trade?",
    trade_analysis=trade_review,
)
print("(B)")
print(answer5b)
print("\n" + "=" * 60 + "\n")

# ── Test 6: Out-of-scope (should return INSUFFICIENT_CONTEXT) ───────
print("TEST 6 — Out-of-scope guardrail test")
print("-" * 50)
answer6 = answer_question(
    question="What is the best crypto exchange to use in 2026?",
    trade_analysis="Looking to open a new exchange account for altcoin futures.",
)
print(answer6)

TEST 1 — Beginner + post-trade CALL option analysis
--------------------------------------------------
DIFFICULTY: beginner 
ANSWER: A CALL option is a financial contract that gives the buyer the right, but not the obligation, to buy an underlying asset at a specified price (strike price) before a specified date (expiration date). In the context of your trade, where you correctly predicted the price increase of the asset from 250.50 to 251.00, a CALL option was the right choice because it allowed you to profit from the anticipated upward price movement. CALL options are suitable when you expect the price of the underlying asset to rise. By using a CALL option, you were able to capitalize on this prediction and turn a $10 investment into a $19.50 payout. This example highlights the importance of selecting the right contract type to match your market outlook and managing risk through defined buy prices. 
CONFIDENCE: high 
SOURCES: none


TEST 2 — Intermediate + pre-trade risk management


In [ ]:
## ------- TEST CASES --------- ##

# Queries a trading-analysis agent would ask from the knowledge base.
# The agent's primary job: analyse charts (screenshots + market values),
# evaluate buy/sell decisions, and ground its reasoning in curated knowledge.

queries = [
    # ── Technical Indicator Interpretation ───────────────────────────
    # (Agent sees indicator values on a chart and needs to understand them)
    "What does it mean when RSI is above 70 and price is near the upper Bollinger Band?",
    "How should I interpret a bearish MACD crossover during a confirmed downtrend?",
    "What does it indicate when price breaks below the 50-period SMA while the EMA remains above?",
    "How do Bollinger Bands help identify potential reversal points on a chart?",
    "At what RSI level is an asset considered oversold and potentially ready for a buy entry?",
    "What is the significance of a bullish divergence between RSI and price action?",
    "How do I confirm a trend reversal using moving average crossovers (golden cross / death cross)?",
    "What does it mean when MACD histogram bars are shrinking while price is still rising?",

    # ── Risk Management & Position Sizing ───────────────────────────
    # (Agent needs to calculate risk before executing a trade)
    "What is the recommended percentage of account balance to risk on a single trade?",
    "How should a stop-loss be placed relative to the nearest support level?",
    "What is the minimum risk-reward ratio I should target before entering a trade?",
    "How does a trailing stop work and when should it be activated?",
    "What position sizing strategy limits drawdown for a $10,000 trading account?",
    "When should I use a fixed stop-loss vs a trailing stop?",

    # ── Market-Specific Analysis (grounded in blog articles) ────────
    # (Agent needs macro context to decide whether to enter or exit)
    "What is the current outlook for gold prices given recent US jobless claims data?",
    "Why did Bitcoin drop 40% and what are analysts saying about further downside risk?",
    "How does the Federal Reserve's rate decision typically affect precious metals like gold and silver?",
    "What caused Amazon stock to plunge after earnings and what does it signal for the tech sector?",
    "What is driving silver's recent 30% price crash and its ripple effect on global markets?",
    "What is the S&P 500 liquidity outlook and how does tightening liquidity affect equity indices?",
    "How are US-China trade tensions influencing current market volatility?",
    "How has oil price volatility historically shaped global equity and commodity markets?",
    "Is the current gold and silver bounce back sustainable or a dead-cat bounce?",
    "What does Microsoft's Azure revenue miss reveal about the AI trade going forward?",
    "Why is Bitcoin struggling to hold $90K while gold and oil surge?",

    # ── Chart-Based Decision Support ────────────────────────────────
    # (Agent sees a specific pattern on a chart and needs actionable guidance)
    "Price just touched the lower Bollinger Band with a spike in volume — is this a buy signal?",
    "RSI shows bullish divergence while price makes new lows — what trade action does this suggest?",
    "The chart shows a death cross (50 SMA crossing below 200 SMA) — should I exit long positions?",
    "Volume is declining while price is making higher highs — is this a bearish warning?",
    "How should I interpret a hammer candlestick pattern at a key support level?",
    "Price has consolidated inside narrowing Bollinger Bands for several days — what usually follows?",

    # ── Strategy & Trade Execution ──────────────────────────────────
    # (Agent choosing between strategies or trade parameters)
    "What are the key differences between day trading and swing trading approaches?",
    "How should I adjust my trading strategy during periods of extreme market volatility?",
    "When should I prefer a limit order over a market order for trade entry?",
    "What factors should I evaluate before entering a leveraged derivatives trade?",
    "How does the Martingale strategy work and what are its risks?",
    "What is the 1-3-2-6 trading strategy and when is it appropriate?",
    "Why is gold considered a hedge against inflation and how does this affect trading decisions?",

    # ── Fundamental + Technical Combined ────────────────────────────
    # (Agent needs to weigh macro data against chart signals)
    "How do macroeconomic indicators like non-farm payrolls affect gold's technical setup?",
    "When earnings reports conflict with technical chart signals, which should take priority for trade entry?",
    "How does rising inflation historically impact equity index technical patterns?",
    "What happens to stock markets when interest rates increase — and how should chart analysis adapt?",

    # ── Out-of-scope (should return INSUFFICIENT_CONTEXT) ──────────
    # (Validates that the RAG pipeline refuses to hallucinate)
    "What is the best cryptocurrency exchange to open a new account on?",
    "How do I file taxes on my trading profits in the United Kingdom?",
    "What programming language is best for building a trading bot from scratch?",
    "Can you recommend a specific stock ticker to buy right now?",
    "What is the current CEO of Deriv's net worth?",
]

print(f"Prepared {len(queries)} test queries\n")

# Preview grouped
for i, q in enumerate(queries, 1):
    print(f"  {i:2d}. {q}")